In [ ]:
import pymannkendall as mk
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.path as mplPath
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib as mpl
from matplotlib.colors import ListedColormap

In [ ]:
dh = xr.open_dataset('D:/Folder_2023/Research/Joan/urban_hist1.nc')
#dh               
dh.data_vars

In [ ]:
fig = plt.figure(figsize=[6,6])
# 111 means 1 row, 1 col and index 1
ax = fig.add_subplot(111, projection=ccrs.PlateCarree(central_longitude=0))
ax.set_extent([27.25, 44.75, -11.75, 6.75], crs=ccrs.PlateCarree())
dh['urban'].sel(time='2014-01-01 00:00:00').plot(ax=ax, cmap=plt.cm.RdYlBu,
                        extend='both',transform=ccrs.PlateCarree())
#levels=np.arange(0, 150, 20),
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linewidth=1)
gl=ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.01, linestyle='--')
gl.xlabels_top = False; gl.ylabels_left = True; gl.ylabels_right=False; gl.xlines = True
gl.xformatter = LONGITUDE_FORMATTER; gl.yformatter = LATITUDE_FORMATTER
plt.title('2014-01-01 00:00:00',fontsize=11, fontweight='bold')
#plt.savefig('C:/ERSSTv5/output/V wind component diff.png')
plt.show()

In [ ]:
###For Annual Trends Only
dat_samp =dh['urban'].groupby('time.year').mean('time')    ## Group data into yearly means for annual trends
#Data

In [ ]:
#dat_samp = Data.sel(longitude=slice(27.25, 44.75), latitude=slice(-11.75, 6.75))  ## Slice to EAC
slope_val = np.zeros((len(dat_samp.latitude.values),len(dat_samp.longitude.values)))
p_value = np.zeros((len(dat_samp.latitude.values),len(dat_samp.longitude.values)))
#output = []
for i in np.arange(len(dat_samp.latitude.values)):
    for j in np.arange(len(dat_samp.longitude.values)):
        
        try:
            slope_val[i,j] = mk.original_test(dat_samp[:,i,j]).slope  ## trend,h,p,z,tau,s,var_s,slope,intercept = mk.original_test(x,0.05)
            p_value[i,j] = mk.original_test(dat_samp[:,i,j]).p
        except:
            slope_val[i,j] = np.nan
            p_value[i,j] = np.nan
            
        #output.append(slope_val)

In [ ]:
## Define data as Xarray dataset and save as netcdf
output1=xr.DataArray(slope_val, dims=('latitude', 'longitude'), coords={'latitude':dat_samp.latitude, 'longitude':dat_samp.longitude}, attrs=dict(description="slope", units="sst year-1"),)
data1 = output1.rename("trend")

output2=xr.DataArray(p_value, dims=('latitude', 'longitude'), coords={'latitude':dat_samp.latitude, 'longitude':dat_samp.longitude}, attrs=dict(description="significance",),)
data2 = output2.rename("p_val")
#data2
## Save Data as netcdf
#data1.to_netcdf('C:/Diabatic_data/Af_CNHR_1991-2021_Annual_trend.nc', mode='w')
#data2.to_netcdf('C:/Diabatic_data/Af_CNHR_1991-2021_Annual_pvalue.nc', mode='w')
#data2.plot()

In [ ]:
## clip the data with the mask
trnd = data1
pval = data2
#pval

In [ ]:
#Plot the map
none_map = ListedColormap(['none'])
fig = plt.subplots(constrained_layout=True, figsize=(12, 10))
mpl.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.linewidth'] = 0.4
plt.gcf().subplots_adjust(hspace=0, wspace=0.08)
ax = plt.subplot(projection=ccrs.PlateCarree(central_longitude=0))

gl=ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.01, linestyle='--')
gl.xlabels_top = False; gl.ylabels_left = True; gl.ylabels_right=False; gl.xlines = True
gl.xformatter = LONGITUDE_FORMATTER; gl.yformatter = LATITUDE_FORMATTER

## Change this to the threshold you need.
cond = (pval <= 0.05)
## Mask out the areas that do not satisfy the conditions
sig_reg = pval.where(cond)

cs = plt.contourf(dat_samp.longitude, dat_samp.latitude, trnd.values[:,:],  
               levels=np.arange(-0.4, 2.1, 0.4),cmap=plt.cm.RdYlBu, extend='both') 
#levels=np.arange(-0.4, 2.1, 0.4)
## make a hatch of significance
hatch = ax.pcolor(dat_samp.longitude, dat_samp.latitude, sig_reg.data[:,:], cmap=none_map, 
                 hatch='...', edgecolor='black', lw=0, zorder=4)

# Adding geographical features
ax.coastlines(resolution='10m', color='black', linewidth=1)
ax.add_feature(cfeature.BORDERS, color='black', linewidth=1)
## title attributes
plt.title('OND season Rainfall Trends (1990-2020)',fontsize=16, fontweight='bold')
plt.xticks(size = 10, fontweight='bold')
plt.yticks(size = 10, fontweight='bold')
##labling the axis
plt.ylabel('Latitude',fontsize=14, fontweight='bold')
plt.xlabel('Longitude',fontsize=14, fontweight='bold')
##ploting the legend
plt.subplots_adjust(bottom=0.08, right=0.8, top=0.8)

###plt.colorbar(cax=cax)
plt.colorbar(cs, ax=ax,cax = plt.axes([0.77, 0.1, 0.018, 0.6]), label='seasonal trends (mm/year)')
# save output
#plt.savefig('F:/Folder_2023/Ingeri/CRU-OND-season-wet_EAC.jpeg', dpi=300)
plt.show()